In [13]:
%load_ext autoreload
%autoreload 2

from NewsContent import *
from UserContent import *
from preprocessing import *
from PEGenerator import *
import PEGenerator
from models import *
from utils import *
from Encoders import *

import os
import numpy as np
import json
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
data_root_path = "../data/Challenge/"
embedding_path = "../../"
KG_root_path = "../data/Challenge/entity"
popularity_path = "../data/Challenge/popularity"
config = {'title_length':30,
              'body_length':100,
              'max_clicked_news':50,
              'npratio':1,
              'news_encoder_name':"CNN",
              'user_encoder_name':"Att",
             'attrs':['title', 'vert', 'topic'],
             'word_filter':0,
             'data_root_path':data_root_path,
             'embedding_path':embedding_path,
             'KG_root_path':KG_root_path,
            'popularity_path':popularity_path,
            'batch_size': 32,
             'max_entity_num':5,
             'max_topics_num': 5,
             'image_emb_len': 1024}
model_config = {
        'news_encoder':0,
        'popularity_user_modeling':True,
        'rel':True,
        'ctr':True,
        'content':True,
        'rece_emb':True,
        'activity':True

    }

In [16]:
News = NewsContent(config)

TrainUsers = UserContent(News.news_index,config,'train.tsv',2)
ValidUsers = UserContent(News.news_index,config,'val.tsv',1)
TestUsers = UserContent(News.news_index,config,'test.tsv',2)

24724
12678
12678


In [17]:
News.fetch_news(np.array([2]))[0]

array([ 7,  8,  9, 10, 11, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 17,  0,  0,  0,
        0,  0])

In [18]:
train_sess,train_buckets, train_user_id, train_label = get_train_input(TrainUsers.session,News.news_index,config)
test_impressions, test_userids = get_test_input(TestUsers.session,News.news_index)
val_impressions, val_userids = get_test_input(ValidUsers.session,News.news_index)

24724
24888
12678
12678


In [19]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path,News.word_dict)
entity_embedding_matrix, have_word2 = load_matrix(embedding_path, News.entity_dict)
topic_embedding_matrix, have_word3 = load_matrix(embedding_path, News.topic_dict)

In [20]:
train_loader = DataLoader(TrainDataset(News, TrainUsers, train_sess, train_user_id, train_buckets, train_label), config['batch_size'])
val_user_data = UserDataset(News,ValidUsers)
test_user_data = UserDataset(News,TestUsers)
news_data = NewsDataset(News)

In [23]:
with open(config['data_root_path']+'image_embeddings.pkl', 'rb') as f:
    data = pickle.load(f)

    image_emb_matrix = np.zeros((len(News.news_index)+1, len(data.popitem()[1]),),dtype='float32')

    for key in data:

        if key in News.news_index:
            image_emb_matrix[News.news_index[key]] = data[key]


In [32]:
from torch.optim import Adam

model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = \
create_pe_model(config, model_config, News, title_word_embedding_matrix, entity_embedding_matrix, image_emb_matrix, topic_embedding_matrix, device)

model.to(device)
for name, module in model.named_children():
    module.to(device)
    print(f'Moved {name} to {device}')



Moved news_encoder to cuda
Moved bias_news_encoder to cuda
Moved pop_aware_user_encoder to cuda
Moved bias_scorer to cuda
Moved activity_gater to cuda
Moved time_embedding_layer to cuda
Moved time_distributed1 to cuda
Moved time_distributed2 to cuda
Moved time_distributed3 to cuda
Moved scaler to cuda
Moved softmax to cuda


In [25]:
colds = [5, 7, 10, 15]
topKs = 10

In [34]:
from tqdm import tqdm
val_metrics_epoch = []
num_epochs = 10
# Step 2: Create your Adam optimizer
optimizer = Adam(model.parameters(), lr=0.0001)

loss_fn = nn.CrossEntropyLoss()

# Step 3: Iterate over the data for the number of epochs
for epoch in range(num_epochs):

# Step 4: Iterate over each batch of data and compute the scores using the forward pass of the network
    model.train()
    for x, y in tqdm(train_loader):
        optimizer.zero_grad()
        x= [i.to(device) for i in x]
        y = y.to(device)
        out = model(x)
    
        # Step 5: Compute the lambda gradient values for the pairwise loss (spedup) with the compute_lambda_i method on the scores and the output labels
        loss = loss_fn(out, y)

        # Step 6: Bacward from the scores with the use of the lambda gradient values
        if loss is not None:
            # torch.autograd.backward(out, loss)
            loss.backward()
            
            # Step 7: Update the weights using the optimizer
            optimizer.step()

    model.eval()

    val_metrics = eval_model(model_config, News, user_encoder, val_impressions, val_user_data, val_userids,
               news_encoder, bias_news_encoder, activity_gater, time_embedding_layer, 
               bias_content_scorer, scaler, colds, topKs, ValidUsers, device)
    
    print("epoch: {}, metrics: {}".format(epoch, val_metrics))
    val_metrics_epoch.append(val_metrics)

    if epoch > 1:
        if (val_metrics_epoch[-1][0] < val_metrics_epoch[-2][0]):
            break



  3%|▎         | 27/778 [00:09<04:24,  2.84it/s]


KeyboardInterrupt: 

In [35]:
colds = [5, 7, 10, 15]
topKs = 10
test_metrics, test_cold_metrics, test_topics, test_ILADs, test_ILMDs = eval_model(model_config, News, user_encoder, test_impressions, test_user_data, test_userids,
               news_encoder, bias_news_encoder, activity_gater, time_embedding_layer, 
               bias_content_scorer, scaler, colds, topKs, TestUsers, device)

  0%|          | 0/12678 [00:00<?, ?it/s]c:\Users\tangs\Documents\uni_stuff\RecSys\RecSys-PP-Rec\Code\PEGenerator.py:266: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  user_feature = torch.IntTensor([self.News.fetch_news(clicked_ids)])
 25%|██▍       | 3158/12678 [01:38<04:55, 32.17it/s]


KeyboardInterrupt: 

In [ ]:
print(News.category_dict)

{'biler': 0, 'om_ekstra_bladet': 1, 'haandvaerkeren': 2, 'plus': 3, 'musik': 4, 'sport': 5, 'nyheder': 6, 'opinionen': 7, 'forbrug': 8, 'nationen': 9, 'bibliotek': 10, 'services': 11, 'dagsorden': 12, 'sex_og_samliv': 13, 'penge': 14, 'krimi': 15, 'play': 16, 'vin': 17, 'ferie': 18, 'auto': 19, 'podcast': 20, 'incoming': 21, 'horoskoper': 22, 'underholdning': 23, 'side9': 24}


In [ ]:
print("val metrics", val_metrics_epoch)
print("test metrics", test_metrics)
print("cold", test_cold_metrics)
print("diversity", [test_ILADs, test_ILMDs])

results = {"config": config,
           "model_config" : model_config,
            "val metrics": val_metrics_epoch, 
           "test metrics": test_metrics,
           "cold": test_cold_metrics,
           "diversity": [test_ILADs, test_ILMDs]}

import json
with open('results{}.json'.format(time.time()), 'w') as f:
    json.dump(results, f)

val metrics []
test metrics [0.638240975012527, 0.4344798063475666, 0.48158064929225997, 0.5378331465271885]
cold [[nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan]]
diversity [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]


## Finished
## ---